In [1]:
include("PSDiGraph.jl")
using .PSDiGraphs
include("dbn_distributions.jl")
include("../simulation-study/data_simulation.jl")
include("dbn_preprocessing.jl")

vectorize_X (generic function with 1 method)

In [2]:
ts_data_path = "data/mukherjee_data.csv"
protein_names_path = "data/protein_names.csv"
prior_adj = "data/prior_graph.csv"
timesteps_path = "data/time.csv"
ts_vec, proteins, parent_sets, timesteps = hill_2012_preprocess(ts_data_path, protein_names_path, prior_adj, timesteps_path);
Xminus, Xplus = combine_X(ts_vec);

In [3]:
function prior_prob(ps, lambda, ps_ref)
    p = exp( -lambda * sum((ps .== true) .& (ps_ref .== false)) )
    return p
end

"""
DOES NOT INCORPORATE PROPOSAL PROBABILITIES.
"""
function approx_acceptance_prob(ind, ps_new, ps_old, lambda, ps_ref, X_plus, X_minus, regression_deg)
    
    println("P(X|G'):")
    ML_Gp = exp(log_marg_lik(ind, ps_new, Xminus, Xplus[:,ind], regression_deg, 1.0))
    p_Gp = prior_prob(ps_new, lambda, ps_ref)
    println("P(G')\t", p_Gp)
    
    println("P(X|G):")
    ML_G = exp(log_marg_lik(ind, ps_old, Xminus, Xplus[:,ind], regression_deg, 1.0))
    p_G = prior_prob(ps_old, lambda, ps_ref)
    println("P(G)\t", p_G)
    
    println("\nPROB RATIO:\t", p_Gp*ML_Gp / (p_G * ML_G))
end

approx_acceptance_prob

In [4]:
parent_sets

20-element Array{Array{Bool,1},1}:
 [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0]
 [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0]
 [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
 [1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1]
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
 [1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1]
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 

In [28]:
clear_caches()
ind = 1
#ps_old = copy(parent_sets[ind])
ps_old = [false for i=1:length(parent_sets[ind])]
#ps_old[ind] = true
println(ps_old)

ps_new = copy(ps_old)
ps_new[15] = true
# ps_new[5] = false
println(ps_new)

lambda = 3.0
ps_ref = parent_sets[ind]

regression_deg = 4

approx_acceptance_prob(ind, ps_new, ps_old, lambda, ps_ref, Xplus, Xminus, regression_deg)

Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
P(X|G'):
	Marg. Like. PART 1:	0.18569533817705186
		29.0	^	-0.5
	Marg. Like. PART 2:	7.994120140933893e-16
		11.977702542938458	^	-14.0
	Marg. Like. TOTAL:	1.4844708429987003e-16
P(G')	1.0
P(X|G):
	Marg. Like. PART 1:	1.0
		29.0	^	-0.0
	Marg. Like. PART 2:	9.139181488855439e-21
		27.00000000000001	^	-14.0
	Marg. Like. TOTAL:	9.139181488855439e-21
P(G)	1.0

PROB RATIO:	16242.929903612305


In [ ]:
using LinearAlgebra

In [ ]:
(dot(Xplus[:,3], Xplus[:,3])*(1.0 - 28.0/29.0))^(-14)

In [53]:
function boolvec_to_ints_1(boolvec::Vector{Bool})
    return [i for (i,b) in enumerate(boolvec) if b] 
end
            
function boolvec_to_ints_2(boolvec::Vector{Bool})            
    return findall(identity, boolvec)
end
            
function generate_boolvec(N::Int64)
    return rand([true,false], N)            
end         

generate_boolvec (generic function with 1 method)

In [54]:
function timeit(fn::Function, args::Tuple, repetitions::Int64)
    st1 = time()
    for i=1:repetitions
        fn(generate_boolvec(args...))
    end
    end1 = time()
    t1 = end1 - st1
    return t1
end

timeit (generic function with 1 method)

In [111]:
timeit(boolvec_to_ints_1, (1000,), 10000)

0.14391779899597168

In [112]:
timeit(boolvec_to_ints_2, (1000,), 10000)

0.13225984573364258

In [51]:
rand([true,false], 10)

10-element Array{Bool,1}:
 1
 1
 1
 0
 0
 1
 0
 0
 1
 1

In [113]:
using Gen

In [115]:
res = searchsorted([1,2,3,4,5], 2.5)

3:2

In [118]:
typeof(res)

UnitRange{Int64}

In [131]:
function ith_nonparent(i, parents::Vector{Int64})
    p = 1
    while parents[p] - p < i
        p += 1
    end
    return i + p - 1
end

ith_nonparent (generic function with 1 method)

In [133]:
ith_nonparent(8,[7,8,10,20])

11